---
title: '<span class="dark-highlight">OpenAstrocytes</span>'
subtitle: '<span class="dark-highlight">AI-ready dynamic activity from the astrocyte network</span>'

title-block-banner: img/astro-banner.png
author-title: ''
affiliation-title: ''
published-title: 'Latest (v1.1)'
# modified-title: 'Updated'

author:
    - name: Maxine Levesque
      email: maxine@forecast.bio
    #   affiliation: " "

    - name: Kira Poskanzer
      email: kira@forecast.bio

    - name: " "

date: 21 November 2025
# date-modified: 31 October 2025

toc-title: "OpenAstrocytes"

fig-format: svg
filters:
  - inline-svg

reference-location: margin
# cap-location: margin
# citation-location: margin
bibliography: references.bib
csl: csl/science.csl

execute:
    # freeze: true
    cache: true
---

## tl;dr

Astrocytes make up about a third of the cells in your brain, and form a second, distinct brain network from the more-studied neuronal network. As neuroscience has discovered in only the last few years, the astrocyte network carries out extremely important computations, integrating information from across different brain networks, and using this information to drive feedback to neurons that controls physiological state and maintains long-term homeostasis. To help push the field's understanding of astrocytes' unique role forward, we're releasing **OpenAstrocytes**---the largest repository of structured astrocyte calcium dynamics to date, drop-in ready for use in AI workflows.

Take a look:

```bash
pip install astrocytes

# or, with `uv`,
uv add astrocytes
```

 

---

## Background

The human brain is about one-third neurons. But two-thirds of it is made from other, *non-neuronal* cells; and, in recent years, the neuroscience field has become more acutely aware of the incredible importance of the roles played by those other players.

One of those other thirds is made up of **astrocytes**---a crucially important subdivision of the brain's little-understood *glia*^[From "glue", referencing old theories of their supposed passivity in brain function.]. Astrocytes form a cellular network of their own, woven between the neurons (@fig-synapse-diagram); and, as recent work has demonstrated, this network encodes rich information about both local and global biochemical *milieu* and physiological state [@cahill2024network].

::: {#fig-synapse-diagram .column-body}
![](img/synapse-diagram.png)

Schematic diagram of the *tripartite synapse*. Astrocytic processes (green) wrap around most cortical synaptic connections between neurons, allowing astrocytes to integrate the myriad chemical signals sensed in the neuronal chatter, and downstream, to incorporate that information with other signals about the local brain network sensed from other parts of the extracellular space.
:::

All biological cells must incorporate a massive number of data-points, coming both from the outside world and also from the internal processes of the cell itself; indeed, this binding-together of disparate interactions into a shared, coherent, soliton-like packet is, perhaps, what makes life *life* [@pezzulo2024active]. While all the diverse kinds of cells across the entire tree of life have unique approaches to sensing and making sense of these various signals, because of shared evolutionary history, a few common structural threads span vast swaths of this diversity. One of those central commonalities is the use of intracellular *calcium signaling*: because of the calcium ion's unique physical properties, cells very early in the evolutionary history of life developed machinery that allowed them to maintain *extremely* low concentrations of calcium within themselves, thereby giving bursts of calcium inside the cell a very robust signal-to-noise ratio. And, like many contingent solutions of evolution, the machinery stuck around!

But astrocytes have a unique---and incredibly rich---way of signaling with calcium; the best way to appreciate it is to look at it by eye:

::: {#fig-astro-video .column-screen}
{{< video img/astro-events.mov >}}

(**Left**) Dynamics of astrocyte calcium, as recorded with two-photon imaging of mouse cortical slices expressing the calcium-binding fluorescent indicator GCaMP. (**Right**) Segmentation of this activity into discrete events using the AQuA toolbox. Reproduced from @wang2019accurate.
:::

As is evident in the video above, naturalistic astrocyte calcium activity is composed of discrete calcium *events*---high-dimensional, complex, activity that is dynamically coupled to both neuronal activity and activity within the rest of the astrocyte network. These sparkles constantly fly across the entire connected astrocyte network, and amazingly, the full details of what lies in the essential chatter of this third of the brain is still under active investigation.

While recent work using more classical ML techniques has gotten us closer to cracking this "calcium code" in the astrocyte network, the rise of today's AI tools raises the possibility of being able to find so many more features in this rich imaging that we, by eye, could never have even conceived of. Unfortunately, much of the data that would be useful in attacking this problem is either locked away or formatted in such a fashion to strongly obfuscate the relevant features for AI training and inference for any non-subspecialist. What's needed to accelerate the field's ability to dig deeper into the structure of this unique form of biological signaling is a data repository designed from the ground up to take advantage of these new tools.

 

---

## Architecture

The best biological data science today is heavily leveraging the advances that have been made in AI over the last few years, and taking advantage of these tools to understand the latent structure of new biological datasets. We have previously used advances in machine learning to get at new features of our data that have led us to exciting new biology [@cahill2024network]; but harnessing the full power of the methods that have risen to prominence over the last few years---and the new infrastructure and logistical challenges they present---has necessitated a ground-up reconceptualization of how we work with data at Forecast.

In particular, we wanted to take advantage of both:

1. the ease and flexibility provided by **streaming data** for prototyping and deploying AI workflows; and,
2. the richness of **structured data** that allows us to quickly make sense of the different metadata elements for samples or batches streamed in.

The former of these points is crucial for the nuts and bolts of scaling model training and inference, and has motivated advances in data tooling for AI like WebDatasets [@aizman2020highperformanceiolarge]. The latter, we believe strongly, is a crucial point of departure for biomedical data; while immense progress has been made across domains by taking advantage of the incredible empirical power that lies in scaling, we believe architecting workflows at their foundation around keeping track of the richness of data's explicit structure---and, most crucially, the structured relationships *between* those explicit data structures [@nlab:vertical_categorification]---is a central part of how to leverage the power of these methods for biological applications, where the intrinsic squishy messiness makes purely *tabula rasa* inference of latent structure much more of an uphill battle.

### Backbone: Distributed, typed WebDatasets with `atdata`

To realize this, we built the OpenAstrocytes dataset on [`atdata`](https://github.com/foundation-ac/atdata), a new open-source package for structured, distributed, and interoperable streaming datasets we're developing at Forecast to aid in AI training and inference, both in our own work and for the larger investigative community.[The original motivation for `atdata` internally was to have fast infrastructure for speeding up our AI iteration cycle when putting together the rich, multimodal datasets we encounter in biomedical R&amp;D; we're excited about the [features we're working on](#sec-future-atdata) rolling out to the community soon!]{.aside}

`atdata` is built on the open [WebDataset](https://github.com/webdataset/webdataset) format, designed for use as a high-performance AI I/O layer used by many other research groups, and so is able to take advantage of their streaming, caching, smart batching, and drop-in PyTorch connections. `atdata` adds functionality that makes it straightforward to understand structured data from remote repositories:

In [ ]:
#| echo: false

import sys
from copy import copy

from tqdm import tqdm

import numpy as np
from numpy.typing import NDArray

import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from forecast_style import HouseStyle

import atdata
import astrocytes

In [ ]:
#| echo: false

# Plotting helpers

def plot_micrograph( s: HouseStyle, image: NDArray,
            ax: Axes | None = None,
            scale_x: float = 1.,
            scale_y: float = 1.,
            scale_bar: float = 100.,
            **kwargs
        ):

    s.show_micrograph( ax, image,
        scale_x = scale_x,
        scale_y = scale_y,
        **kwargs
    )
    # plt.imshow( sample_view.image[0] )
    s.label(
        data_xlim = (0, scale_bar),
        data_ylim = None,
    )
    plt.xticks( [0, scale_bar], ['0', '100 $\\mu$m'], ha = 'left' )
    plt.yticks( [] )

In [ ]:
#| label: fig-example-image
#| fig-column: body-outset-right
#| fig-cap: Example data from the OpenAstrocytes dataset, showing mean calcium fluorescence in an *ex vivo* brain slice from a mouse over 60 frames (about one minute).
import astrocytes

# Dataset sample type schema
from astrocytes.schema import BathApplicationFrame

dataset = astrocytes.data.bath_application \
            .as_type( BathApplicationFrame )

# Pop off a single sample
exemplar = next( x for x in dataset.ordered( batch_size = None ) )

# Pop off and z-project a single batch
batch = next( x for x in dataset.ordered( batch_size = 60) )
batch_summary = np.quantile( batch.image[:, 0], 0.75, axis = 0 )

with HouseStyle() as s:
    fig, ax = plt.subplots( figsize = (8, 8) )
    plot_micrograph( s, batch_summary, ax,
        scale_x = exemplar.scale_x,
        scale_y = exemplar.scale_y,
        scale_bar = 100.,
        clim = (0, 140),
    )

Here, the `as_type` method is hiding logic under the hood that takes advantage of pre-implemented bidirectional [lenses](https://ncatlab.org/nlab/show/lens+%28in+computer+science%29) that are opinionated about the way that generic microscopy images should be interpreted, if possible, as data coming from the specific experiments in the OpenAstrocytes data. These lenses have a [rich compositional structure](https://ncatlab.org/nlab/show/Kleisli+category), and we are excited about the possibilities that follow downstream from starting at bedrock by keeping track of the full semantics of underlying datasets (that humans worked hard to put together!), as well as the ways they [fit together](https://ncatlab.org/nlab/show/2-category).

The full documentation for `atdata` can be found on [GitHub](https://github.com/foundation-ac/atdata).

### Datasets

Our initial release of OpenAstrocytes is built from two large collections of previously-published calcium imaging from mouse astrocytes, originally analyzed in @reitman2023norepinephrine and @cahill2024network. In total, these datasets comprise in total about 1M images, making this release among the the largest open datasets of astrocyte calcium dynamics to date, to our knowledge.

This initial release of the OpenAstrocytes corpus contains two-photon imaging of astrocyte intracellular calcium. These data were recorded during two different experimental setups:

1. *Bath application* (`astrocytes.data.bath_application`): in these experiments, *ex vivo* visual cortical slices from mice were exposed to one of two different agents (baclofen or *t*-ACPD) after a baseline period, with the agents added to the solution bathing the full slice after the time of addition.
2. *Two-photon photo-uncaging* (`astrocytes.data.uncaging`): in these experiments, *ex vivo* visual cortical slices from mice were exposed to one of two caged neurotransmitters (RuBi-GABA or RuBi-glutamate), with a pulse (or pulses) of a second laser applied after a baseline period to "free" the GABA or glutamate from the ruthenium bipyridine cage rendering it inert initially, but only within the small, ~10µm locus directly stimulated by the second laser [@fino2009rubi].

If you want to play around with OpenAstrocytes, you can add it to your Python project with

``` bash
pip install astrocytes

# or, if you're *really* cool 😎,
uv add astrocytes
```

 

------------------------------------------------------------------------

## Demo

Let's take a look at some of the fun structure that we have in the OpenAstrocytes data. Here, we'll be digging into one of the experiments from @cahill2024network, in which *ex vivo* slices from mouse visual cortex had two drugs---baclofen and *t*-ACPD^[Baclofen acts as an agonist of the GABA~B~ receptor; *t*-ACPD acts as an agonist of the metabotropic glutamate receptor subtypes mGluR2 and mGluR3]---bath-applied across the slice. Here, we'll use the OpenAstrocytes dataset with contemporary tools to give views into the differences between these two compounds' effects on brain networks.

At the outset, we'll tuck away any parameters for this experimental setup that will come up across analyses.[Click any of the chevrons to expand the code that generates figure panels in this pub.]{.aside}

In [ ]:
#| code-fold: true
#| output: false

# You found me, the parameter!
t_intervene = 300
"The applied compound begins to enter the bath at 300s after recording onset"

### Approach

In previous work [@cahill2024network], we identified the differences in astrocyte networks' responses to baclofen and *t*-ACPD by using a spatial-temporal segmentation technique known as graphical time warping [@NIPS2016_f0bbac6f], as implemented in the AQuA toolbox [@wang2019accurate]. However, recent advances in large, self-supervised image models like DINOv3 [@siméoni2025dinov3] have greatly expanded our ability to pull out salient, highly-nontrivial latent structures within image data in an unbiased way. As a simple demonstration of this paradigm as applied to the OpenAstrocytes dataset, let's investigate the extent to which this model is able to represent physiologically-relevant information in these specialized biological images---even *without* any post-training.

The overall approach presented in this demo is to run inference on the OpenAstrocytes bath application data using one of the vanilla DINOv3 models (`dinov3-vit7b16-pretrain-lvd1689m`), and leverage the fact that this model's internal architecture separates out its image feature representations into *patch embeddings*. Each microscopy image is separated by the model into a 14-by-14 grid of image patches, with each patch corresponding to a ~60µm square portion of the imaged slice. At each time point over the course of an individual recorded movie, DINOv3 provides each one of these patches with a time series of 4096-dimensional embeddings, corresponding to its decomposition of the image into representative features as learned from its self-supervision over a vast swath of (principally non-biological, but with some exceptions) images.

Here, we use a simple linear basis change (*via* principal component analysis) to find a 64-dimensional subspace of this 4096-dimensional space already learned within the DINOv3 self-supervision objective that captures the subset of overall image features that appear in individual astrocyte calcium patches. This subspace captures both the image feature variability across different local astrocyte network morphologies as imaged in individual patches (as in @fig-patch-3d-raw), as well as the image feature variability induced by the specific changes of the applied pharmacology in the bath application experiment. As it turns out, the richness in these feature representations---even in the absence of any biology-specific fine-tuning!---is enough to decode the identity of the applied compound from the calcium fluorescence pattern within a single ~60µm patch at a single time-point.

### DINOv3 patch embeddings capture rich, dynamic responses in astrocyte networks

First, we'll examine the latent feature structure of *local* patches of astrocyte network calcium activity, and how those features unfold after application of pharmacology. We're particularly interested in looking at local patches to start because of our prior on the biology: because of a significant and growing body of work elucidating the information content of astrocyte calcium activity on these spatial scales [@lia2021calcium], we have a hint that these features in our observables are most likely paint a generalizable readout of physiologic changes induced by pharmacology.

#### Individual patch responses
Let's download a representative movie from the OpenAstrocytes bath application dataset---the same recording we projected above in @fig-example-image to take a peek at the overall anatomy of the slice---and inspect the behavior of a couple representative patches from it:

In [ ]:
#| echo: false
from astrocytes.schema import BathApplicationFrame

def _get_movie(
            movie_uuid: str,
            verbose: bool = False
        ) -> list[BathApplicationFrame]:
    "Helper for "

    def _vprint( *args, **kwargs ):
        if verbose:
            print( *args, **kwargs )

    #

    ds = astrocytes.data.bath_application \
            .as_type( BathApplicationFrame )

    started_movie = False
    recording_frames = []
    _vprint( 'Iterating frames...' )
    for frame in ds.ordered( batch_size = None ):
        if frame.movie_uuid == movie_uuid:
            _vprint( 'Snagging correct movie...' )
            started_movie = True
            recording_frames.append( frame )
        else:
            if started_movie:
                # We've moved to a new movie uuid, and so finished up
                _vprint( 'Got it!' )
                break
    
    return recording_frames

In [ ]:
#| code-fold: true
#| label: fig-patch-examples
#| fig-column: page-inset-right
#| fig-cap: Two example patches sub-sampled from the same recording as above in @fig-example-image, showing evolution of calcium activity over time relative to bath application of baclofen at $t = 0$. (A small Gaussian smoothing kernel in space and time has been applied here, to aid visualization.)
# Snag a convenient smoother for image preprocessing
from scipy.ndimage import gaussian_filter

## Constants for DINOv3
N_PATCHES_Y = 14
N_PATCHES_X = 14

## Data hyperparameters
test_patches = [
    (10, 12),
    (3, 7),
]
"Indices (vertical, horizontal) of the demo patches to show"

first_movie_id = 'd22f6a65-b3e2-46c2-ba5c-551920af1fe3'
"Microscope-supplied UUID of the demo movie to show"

##

movie_frames = _get_movie( first_movie_id, verbose = False )

first_image = movie_frames[0].image[0]
patch_size_y = first_image.shape[0] / N_PATCHES_Y
patch_size_x = first_image.shape[1] / N_PATCHES_X

test_patch_frames = []
for cur_i, cur_j in test_patches:
    patch_idx_y = ( int( np.floor( patch_size_y * cur_i ) ),
                    int( np.ceil( patch_size_y * (cur_i + 1) ) ) )
    patch_idx_x = ( int( np.floor( patch_size_y * cur_j ) ),
                    int( np.ceil( patch_size_y * (cur_j + 1) ) ) )

    test_patch_frames.append(
        np.array( [ 
            frame.image[0][patch_idx_y[0]:patch_idx_y[1], :][:, patch_idx_x[0]:patch_idx_x[1]]
            for frame in movie_frames
        ] )
    )

test_patch_frames_filtered = [
    gaussian_filter( fs, sigma = (3., 0.6, 0.6) )
    for fs in test_patch_frames
]

## Figure parameters

patch_cmax = 110
"Maximum intensity value for patch snapshot figure"

i_frame_compare = (250, 350, 450, 550, 650, 750)
"Indices of time-slices to display for patch snapshot figure"

#

dt = movie_frames[3].t
n_patches = len( test_patch_frames_filtered )
n_frame_compare = len( i_frame_compare )

##

with HouseStyle() as s:

    fig, axs = plt.subplots( n_patches, n_frame_compare,
        figsize = (10, 4),
        sharey = True,
    )

    for i_patch in range( n_patches ):

        cur_patch_frames = test_patch_frames_filtered[i_patch]

        for i_t in range( n_frame_compare ):

            axs[i_patch, i_t].imshow(
                cur_patch_frames[i_frame_compare[i_t], :, :],
                #
                clim = (0, patch_cmax),
                cmap = 'afmhot',
            )

            if i_patch == 0:
                t_val = i_frame_compare[i_t] * dt - t_intervene
                axs[i_patch, i_t].set_title( f'{t_val:+0.0f}s', fontsize = 12 )

            if i_patch == (n_patches - 1) and i_t == 0:
                axs[i_patch, i_t].tick_params( axis = 'x', length = 3 )
                axs[i_patch, i_t].set_xticks( [-0.5, patch_size_x+0.5], ['0', f'{movie_frames[0].scale_x * patch_size_x:0.0f} µm'], ha = 'left' )
            else:
                axs[i_patch, i_t].tick_params( axis = 'x', length = 0 )
                axs[i_patch, i_t].set_xticks( [-0.5, patch_size_x+0.5], ['', ''] )
            
            axs[i_patch, i_t].set_yticks( [] )

            if i_t == 0:
                axs[i_patch, i_t].set_ylabel( f'Patch {i_patch+1}' )

    fig.patches.extend( [
        plt.Rectangle( (0.382, 0.0995), 0.53, 0.82,
            fill = True,
            color = 'k',
            alpha = 0.05,
            zorder = -1_000,
            transform = fig.transFigure,
            figure = fig,
        )
    ] )

    plt.subplots_adjust( wspace = 0.04, hspace = 0. )

These two patches are good illustrations of the "median" structure of this astrocyte two-photon imaging data. Both patches do exhibit *some* changes after compound application; in comparing, for example, the right four columns with the left two, a close inspection reveals some parts of the patch with evoked activations. But, these changes can be subtle to the unaided or non-specialist eye---particularly here with Patch 2, shown in the bottom row of panels, which we picked to show what a *very* subtle case looks like in raw form.

However, the large image model is remarkably able to pull out a good deal of structure in its latent feature embeddings. We see this best if we first reduce to looking at the particular dimensions of that embedding space maximally capturing the features exhibited in the OpenAstrocytes corpus^[*I.e.*, the top few PCs of the embedding features.]. Specifically, we find that there are axes (PCs) within the patch-embedding space that capture robust changes induced in the astrocyte network as seen through our imaging data---even though those changes, as we saw before, were at first quite subtle to the unaided eye:

In [ ]:
#| code-fold: true
#| label: fig-patch-example-traces
#| column: body-outset-right
#| fig-cap: "For each of the two astrocyte network patches shown above in @fig-patch-examples, each trace shows the time-evolution of one representative embedding PC, relative to the onset of bath application of baclofen at *t*\_=\_0. Time shown as seconds relative to compound entering the bath (shaded box); PC changes shown as multiples of the standard deviation of fluctuations in the pre-application baseline period. (*n.b.*: a 60s boxcar filter in time has been applied to each PC in order to compensate for intrinsic imaging noise, leading to blurring of activation edges.)"

from astrocytes._datasets._embeddings import (
    PatchEmbeddingTrace,
)

## Figure params

baseline_window = (-90, -30)
plot_window = (-100, 240)
panel_verbose = False

panel_yl = (-8, 13)

#

wds_url = (
    'https://data.forecastbio.cloud'
    + '/testing/patch-pc-traces/bath-application/'
    + 'bath_app-dinov3_vit7b16-pca64-smooth84.tar'
)
ds = atdata.Dataset[PatchEmbeddingTrace]( wds_url )

it = ds.ordered( batch_size = None )
if panel_verbose:
    it = tqdm( it )

#

test_traces = [ None for _ in test_patches ]
for trace in it:
    try:
        assert trace.metadata is not None and 'uuid' in trace.metadata
        assert trace.metadata['uuid'] == first_movie_id
        
        for i_patch, (cur_i, cur_j) in enumerate( test_patches ):
            if trace.i_patch == cur_i and trace.j_patch == cur_j:
                test_traces[i_patch] = trace
        
        should_finish = True
        for x in test_traces:
            if x is None:
                should_finish = False
        if should_finish:
            break

    except:
        # Skip traces without needed metadata
        continue

#

def _baseline_normalize( ts, ys, window ):
    filter_cur = (
        (ts >= window[0])
        & (ts < window[1])
    )
    mean = np.mean( ys[filter_cur] )
    std = np.std( ys[filter_cur] )
    return (ys - mean) / std, mean, std

#

causal_offset = (84 * dt) / 4.
"The smoothing window used to postprocess the PCs was 84 frames (~60s)"
t_rel = test_traces[0].ts - t_intervene + causal_offset

with HouseStyle( grids = True ) as s:

    fig, axs = plt.subplots( n_patches, 1,
        figsize = (7, 7),
        sharex = True,
    )

    for i_patch in range( n_patches ):
        ax = axs[i_patch]
        cur_trace = test_traces[i_patch]

        values_0_norm, mean_0, std_0 = \
            _baseline_normalize( t_rel, cur_trace.values[0, :], baseline_window )    
        values_1_norm, mean_1, std_1 = \
            _baseline_normalize( t_rel, cur_trace.values[1, :], baseline_window )
        values_2_norm, mean_2, std_2 = \
            _baseline_normalize( t_rel, cur_trace.values[4, :], baseline_window )

        filter_plot = (
            (t_rel >= plot_window[0])
            & (t_rel < plot_window[1])
        )

        ax.plot( t_rel[filter_plot], values_0_norm[filter_plot],
            'C0-',
            label = 'PC 1',
        )
        ax.plot( t_rel[filter_plot], values_1_norm[filter_plot],
            'C2-',
            label = 'PC 2',
        )
        ax.plot( t_rel[filter_plot], values_2_norm[filter_plot],
            'C3-',
            label = 'PC 5',
        )

        ax.fill_between( [0, plot_window[1]], panel_yl[0], panel_yl[1],
            color = 'k',
            alpha = 0.05,
            linewidth = 0,
        )

        ax.plot( plot_window, [0, 0],
            'k-',
            linewidth = 1,
        )
        ax.plot( [plot_window[0], plot_window[0]], panel_yl,
            'k-',
            linewidth = 1,
        )

        ax.set_xticks( [-90, 0, 60, 120, 180, 240] )
        ax.set_xlim( plot_window )
        ax.set_ylim( panel_yl )

        ax.set_ylabel( f'Patch {i_patch + 1}' )

        if i_patch == 0:
            ax.text( 5, -6.5, '+ Drug',
                va = 'center',
                alpha = 0.5,
            )
        
        if i_patch == (n_patches - 1):
            ax.set_xlabel( 'Time (s)' )
            plt.legend(
                fontsize = 12,
                loc = 'upper left',
            )

    plt.subplots_adjust( hspace = 0.08 )

#### Response heterogeneity across the imaging field
The traces above depict only three of the 64 PCs we computed as our bath application--specific subspace^[These 64 PCs captured a large majority of the observed variance in the 4096-dimensional DINOv3 patch-embedding features. Hence, while our astrocyte two-photon calcium dynamics data is quite rich, it is also the case, as expected, that the statistical structure of these specific sorts of biological images occupies a very small subspace of the full space of naturalistic images.]---and, showed only two of the 196 (14-by-14) patches obtained from inference on this one experimental session alone. Already, we are getting a sense for the detail that the embedding model can pull out from these data.

Indeed, when we look at the feature dynamics across all patches within the imaging field, the full richness of this method in characterizing astrocyte calcium becomes apparent:

::: {.panel-tabset}
##### PC 1

In [ ]:
#| code-fold: true
#| label: fig-patch-example-grid-pc1
#| column: screen-inset
#| fig-cap: Baseline-normalized changes in DINOv3 embedding PC1 features after bath application of baclofen for each of the 196 astrocyte network patches, derived from the same slice depicted above in @fig-example-image. The locations of the two representative patches shown in greater detail in @fig-patch-examples and @fig-patch-example-traces are highlighted in black boxes. Note the heterogeneity of responses across the imaging field, as well as the correspondence between portions of the network exhibiting PC1 activation and anatomical features of the local astrocyte network in @fig-example-image. \newline Time shown as seconds relative to compound entering the bath (shaded box); changes shown as multiples of the baseline distribution standard deviation.

def _get_movie_traces(
            movie_uuid: str,
            verbose: bool = False
        ) -> list[list[PatchEmbeddingTrace]]:
    "Return all patch embedding traces for a given recording"

    def _vprint( *args, **kwargs ):
        if verbose: print( *args, **kwargs )

    # TODO Change over to `atdata`'s new `wids` indexed
    # WebDatasets for faster streaming  in demos
    wds_url = (
        'https://data.forecastbio.cloud'
        + '/testing/patch-pc-traces/bath-application/'
        + 'bath_app-dinov3_vit7b16-pca64-smooth84.tar'
    )
    ds = atdata.Dataset[PatchEmbeddingTrace]( wds_url )

    started_movie = False
    movie_traces: list[PatchEmbeddingTrace] = []

    _vprint( 'Iterating traces...' )
    for trace in ds.ordered( batch_size = None ):
        try:
            assert trace.metadata is not None
            assert 'uuid' in trace.metadata

            if trace.metadata['uuid'] == movie_uuid:
                _vprint( 'Snagging correct movie...' )
                started_movie = True
                movie_traces.append( trace )

            else:
                if started_movie:
                    # We've moved to a new movie uuid,
                    # and so are finished here
                    _vprint( 'Got it!' )
                    break
        except:
            # Skip trace - no metadata
            continue
    
    # Arrange result spatially
    ret = [
        [
            None
            for j in range( N_PATCHES_X )
        ]
        for i in range( N_PATCHES_Y )
    ]

    for trace in movie_traces:
        ret[trace.i_patch][trace.j_patch] = trace

    return ret


##

# Plot params
grids_ylim = (-12.5, 12.5)


##

# Collate full dataset
first_movie_traces = _get_movie_traces(
    first_movie_id,
    verbose = False,
)

# Do plot
i_pc = 0

with HouseStyle() as s:

    fig, axs = plt.subplots( N_PATCHES_Y, N_PATCHES_X,
        figsize = (16, 12),
    )

    for i in range( N_PATCHES_Y ):
        for j in range( N_PATCHES_X ):
            ax = axs[i, j]

            trace = first_movie_traces[i][j]
            ts_rel = trace.ts - t_intervene + causal_offset

            cur_values_norm, _, _ = \
                _baseline_normalize( t_rel, trace.values[i_pc, :], baseline_window )  

            filter_plot = (
                (ts_rel >= plot_window[0])
                & (ts_rel < plot_window[1])
            )

            ax.plot( ts_rel[filter_plot], cur_values_norm[filter_plot],
                'C0-',
                linewidth = 1.5,
            )
            
            xl = ax.get_xlim()
            ax.plot( xl, [0, 0], 'k-', linewidth = 1, zorder = -200 )
            yl = grids_ylim
            ax.fill_between( [0, xl[1]], yl[0], yl[1], color = 'k', alpha = 0.05, linewidth = 0, zorder = 100 )

            if (i, j) in test_patches:
                ax.fill_between( [xl[0], xl[1]], yl[0], yl[1], facecolor = 'none', edgecolor = f'k', alpha = 0.7, linewidth = 2., zorder = -300 )
            
            ax.set_xlim( xl )
            ax.set_ylim( yl )

            ax.set_yticks( [] )
            ax.set_xticks( [] )
    
    for i in range( 1, N_PATCHES_X ):
        axs[-1, i].set_xticks( [-90, 0, 240], ['', '', ''] )
    axs[-1, 0].set_xticks( [-90, 0, 240], ['–90', '0', '240'],
        fontsize = 11,
        rotation = 90,
    )

    axs[-1, 0].set_yticks( [-10, 0, 10], ['-10', '0', '10'],
        fontsize = 11,
    )

    fig.subplots_adjust( wspace = 0.1, hspace = 0.1 )

##### PC 2

In [ ]:
#| code-fold: true
#| label: fig-patch-example-grid-pc2
#| column: screen-inset
#| fig-cap: As in @fig-patch-example-grid-pc1, but for embedding PC2 features. Note that while the distribution of loci that exhibit modulation post-application still respects the underlying anatomy of @fig-example-image, the spatial and temporal distribution is distinct from the changes seen for PC1. Time shown as seconds relative to compound entering the bath (shaded box); changes shown as multiples of the baseline distribution standard deviation.

# Do plot for PC 2
i_pc = 1
grids_ylim = (-18, 18)

with HouseStyle() as s:

    fig, axs = plt.subplots( N_PATCHES_Y, N_PATCHES_X,
        figsize = (16, 12),
        # sharex = True,
        # sharey = True,
    )

    for i in range( N_PATCHES_Y ):
        for j in range( N_PATCHES_X ):
            ax = axs[i, j]
            trace = first_movie_traces[i][j]
            ts_rel = trace.ts - t_intervene + causal_offset

            cur_values_norm, cur_mean, cur_std = \
                _baseline_normalize( t_rel, trace.values[i_pc, :], baseline_window )  

            filter_plot = (
                (ts_rel >= plot_window[0])
                & (ts_rel < plot_window[1])
            )

            ax.plot( ts_rel[filter_plot], cur_values_norm[filter_plot], f'C2-', linewidth = 1.5, zorder = 200 )
            
            xl = ax.get_xlim()
            ax.plot( xl, [0, 0], 'k-', linewidth = 1, zorder = -200 )
            # yl = ax.get_ylim()
            yl = grids_ylim
            ax.fill_between( [0, xl[1]], yl[0], yl[1], color = 'k', alpha = 0.05, linewidth = 0, zorder = 100 )

            if (i, j) in test_patches:
                ax.fill_between( [xl[0], xl[1]], yl[0], yl[1], facecolor = 'none', edgecolor = f'k', alpha = 0.7, linewidth = 2., zorder = -300 )
            
            ax.set_xlim( xl )
            ax.set_ylim( yl )

            ax.set_yticks( [] )
            ax.set_xticks( [] )
    
    for i in range( 1, N_PATCHES_X ):
        axs[-1, i].set_xticks( [-90, 0, 240], ['', '', ''] )
    axs[-1, 0].set_xticks( [-90, 0, 240], ['–90', '0', '240'], fontsize = 11, rotation = 90 )

    for i in range( 0, N_PATCHES_Y - 1 ):
        # axs[i, 0].set_yticks( [-10, 0, 10], ['', '', ''] )
        pass
    axs[-1, 0].set_yticks( [-15, 0, 15], ['-15', '0', '15'], fontsize = 11 )

    fig.subplots_adjust( wspace = 0.1, hspace = 0.1 )

:::

Overall, we see that while the activity of individual astrocyte network patches may be evident-but-subtle to the eye, the large, generalist DINOv3 image model provides a quantitative view of robust changes in calcium imaging features after drug application---a view that captures both the many-dimensional richness of local changes, as well as the heterogeneity of those responses in individual segments of the astrocyte network across an individual cortical slice.

#### Components of static anatomy and dynamic physiology in patch embeddings
Because the original PCA we performed to determine the bath application-–specific subspace of DINOv3 patch embeddings was computed across the full dataset, the image feature variance optimized through PCA necessarily incorporated contributions from two distinct facets of the image structure:

1. the contribution from variations *across* the static astrocyte network anatomical architectures imaged within the various patches; and
2. the physiological dynamics that unfold *within* a single patch.

In order to appreciate the structure of these two distinct contributions within the PC subspace projections in the OpenAstrocytes bath application dataset, we can take a look at a handful of representative patches to visualize the way astrocyte network patches "dance" around PC-space over the course of an experimental session:

In [ ]:
#| echo: false

## Figure parameters

pcs_show = (0, 1, 4)
n_traces_show = 1_000
window_3d = (0, 240)

panel_verbose = False

##

# Collate data

wds_url = (
    'https://data.forecastbio.cloud/testing/patch-pc-traces/bath-application/'
    + 'bath_app-dinov3_vit7b16-pca64-smooth84.tar'
)
ds = atdata.Dataset[PatchEmbeddingTrace]( wds_url )

it = ds.shuffled( batch_size = None )
if panel_verbose:
    it = tqdm( it )

#

i_trace = 0
trace_data_show = []
trace_data_raw_show = []

for trace in it:
    ts_rel = trace.ts - t_intervene + causal_offset
    filter_3d = (
        (ts_rel >= window_3d[0])
        & (ts_rel < window_3d[1])
    )

    cur_trace_pcs_post = []
    for i_pc in pcs_show:
        cur_vals_norm, _, _ = _baseline_normalize( t_rel, trace.values[i_pc, :], baseline_window )  
        cur_trace_pcs_post.append( cur_vals_norm[filter_3d] )
    
    trace_data_show.append( np.array( cur_trace_pcs_post ) )
    trace_data_raw_show.append( trace.values[pcs_show, :] )

    i_trace += 1
    if i_trace >= n_traces_show:
        break
    
trace_data_show = np.array( trace_data_show )
trace_data_raw_show = np.array( trace_data_raw_show )

In [ ]:
#| code-fold: true
#| label: fig-patch-3d-raw
#| column: body-outset
#| fig-cap: "Three-dimensional slice of patch embedding PCs for a random selection of 100 astrocyte network patches as they unfold in time; colors are randomly chosen for each individual patch's dynamics. Black dots indicate the embedding of the network patch when the applied compound enters the bath at *t*\_=\_0; colored dots with black circles indicate the embedding at *t*\_=\_4 min after application, with the colored line between the two dots depicting the embedding-space trajectory of the patch over the course of the recording. Grey lines indicate the PC-space origin. \\newline Among this ~1% subset of patches within the OpenAstrocytes bath application dataset, trajectories are generally separable, indicating contributions of overall differences in underlying astrocytic anatomy from patch to patch. As expected, a number of the patches are *relatively* unresponsive to the applied compound, with only a small degree of motion in embedding-space across the recording (wiggly lines). However, a subset of the patches show strong, directed deviations after compound application, indicating reflections of local physiological modulation in the patch's image statistics."
## Figure parameters
random_seed = 89
n_traces_show_raw = 100

##

rng = np.random.default_rng( random_seed )
traces_show_raw = rng.permutation( n_traces_show )[:n_traces_show_raw]

##

with HouseStyle() as s:
    fig = plt.figure( figsize = (10, 10) )

    ax = fig.add_subplot( projection = '3d' )
    ax.view_init( azim = -62 )

    #

    it = traces_show_raw
    if panel_verbose:
        it = tqdm( it )

    for i_trace in it:
        # trace start
        ax.scatter(
            trace_data_raw_show[i_trace, 0, 0],
            trace_data_raw_show[i_trace, 1, 0],
            trace_data_raw_show[i_trace, 2, 0],
            color = 'k',
            s = 3
        )
        # trace end
        ax.scatter(
            trace_data_raw_show[i_trace, 0, -1],
            trace_data_raw_show[i_trace, 1, -1],
            trace_data_raw_show[i_trace, 2, -1],
            facecolor = f'C{i_trace % 4}',
            edgecolor = 'k',
            linewidth = 0.5,
            s = 8
        )
        # trajectory
        ax.plot(
            trace_data_raw_show[i_trace, 0, :],
            trace_data_raw_show[i_trace, 1, :],
            trace_data_raw_show[i_trace, 2, :],
            f'C{i_trace % 4}-',
            linewidth = 0.5,
            alpha = 0.8,
        )

    ax.set_xticks( [0], '' )
    ax.set_yticks( [0], '' )
    ax.set_zticks( [0], '' )

    ax.set_xlabel( f'PC {pcs_show[0]+1}' )
    ax.set_ylabel( f'PC {pcs_show[1]+1}' )
    ax.set_zlabel( f'PC {pcs_show[2]+1}', rotation = 90 )

    ax.set_xlabel( f'PC {pcs_show[0]+1}', rotation = -17 )
    ax.set_ylabel( f'PC {pcs_show[1]+1}', rotation = 43 )
    ax.set_zlabel( f'PC {pcs_show[2]+1}', rotation = 90 )
    
    ax.set_box_aspect((1, 1, 1.1), zoom=0.92)

As expected, the largest contributor to the heterogeneity of individual frames' latent embeddings appears to come from the differences in *anatomical* structure---that is, from the regularities of the frames' image content arising from the difference in the shape of the cells within the specific patch---as opposed to *physiologic* changes within a static anatomical architecture. We see this in the distinctive separation of the trajectories of individual patches within embedding-space: while the positions of individual patches do change over time across the recording, these dynamic changes tend to be notably smaller relative to the separation *between* full patch trajectories.

However, quite interestingly, those dynamic deflections do *also* appear to possess their own distinct structure. We can emphasize these deviations by plotting the same patch trajectories again, but this time normalized to each individual patch's baseline period:

In [ ]:
#| code-fold: true
#| label: fig-patch-3d-norm
#| column: body-outset
#| fig-cap: Embeddings of the same patches from @fig-patch-3d-raw, but normalized to the fluctuations of each patch within the baseline period before application of pharmacology. Scale shown as multiples of the per-patch baseline standard deviation for each PC. \newline This view highlights that there are a number of patches that are relatively unresponsive to applied pharmacology and remain relatively close to the origin (possibly owing, for example, to heterogeneity in calcium sensor expression)---and at the same time, a number of patches that exhibit extremely large (~50$\sigma$!) excursions.

with HouseStyle() as s:
    fig = plt.figure( figsize = (10, 10) )
    ax = fig.add_subplot( projection = '3d' )
    ax.view_init( elev = 20 )

    # for i_trace in tqdm( traces_show_raw ):
    for i_trace in traces_show_raw:
        # start
        ax.scatter(
            trace_data_show[i_trace, 0, 0],
            trace_data_show[i_trace, 1, 0],
            trace_data_show[i_trace, 2, 0],
            color = 'k',
            s = 3,
        )
        # end
        ax.scatter(
            trace_data_show[i_trace, 0, -1],
            trace_data_show[i_trace, 1, -1],
            trace_data_show[i_trace, 2, -1],
            facecolor = f'C{i_trace % 4}',
            edgecolor = 'k',
            linewidth = 0.5,
            s = 8,
        )
        # trajectory
        ax.plot(
            trace_data_show[i_trace, 0, :],
            trace_data_show[i_trace, 1, :],
            trace_data_show[i_trace, 2, :],
            f'C{i_trace % 4}-',
            linewidth = 0.5,
            alpha = 0.8,
        )
    
    ax.set_xlim( -32, 32 )
    ax.set_ylim( -32, 32 )
    ax.set_zlim( -12, 52 )

    # `pyplot` 3d ticks are a bit too loose!
    ax.tick_params(
        axis = 'y',
        which = 'major',
        pad = -5,
    )

    ax.set_xticks( [-30, 0, 30], ['', '', ''],
        fontsize = 12,
    )
    ax.set_yticks( [-30, 0, 30], ['–30', '', '+30'],
        rotation = -55,
        ha = 'left',
        fontsize = 12,
    )
    ax.set_zticks( [-10, 0, 50], ['–10', '', '+50'],
        fontsize = 12,
    )

    ax.set_xlabel( f'PC {pcs_show[0]+1}',
        rotation = -14 )
    ax.set_ylabel( f'PC {pcs_show[1]+1}',
        rotation = 37 )
    ax.set_zlabel( f'PC {pcs_show[2]+1}',
        rotation = 90 )
    
    ax.set_box_aspect( (1, 1, 1.1),
        zoom = 0.87,
    )

Put together, these results indicate that the latent structure of patch embeddings from DINOv3---a large, self-supervised image model without *specific* biological emphasis during training---contains subspaces rich in information about both the anatomical structure of cells within the astrocyte network (as captured through two-photon imaging of our particular intracellular calcium indicator). This is a remarkable testament to both the intricate order of the compositional structure within image data writ large, as well as to the remarkable power that the self-supervised learning objective has in yielding models able to internalize many highly-nontrivial features of this intricate structure within the statistics of naturalistic images, even within highly-specific domains.


### Astrocyte patch embeddings linearly separate physiological impacts of distinct compounds

The amazing richness of these AI-generated embeddings' portrait of physiologic changes within local network patches after the application of pharmacology---even if not the *largest* contributor *per se* to the structure of embeddings within a vanilla image model like DINOv3---raises the question of whether a small subspace of imaging features contains information about the identity of the applied compound. This would extend our prior work demonstrating that specific features of astrocyte calcium dynamics, derived from more classical ML techniques, encode this sort of information at the *network* level, aggregating across full imaging fields [@cahill2024network]. The extent to which these sorts of physiologic encodings are present within individual small network patches---and the intricacies of such encodings' structure---have not yet been fully elucidated.

#### Individual patch responses
To investigate the extent of these local differences, we can first look at one of the same representative patches we took a gander at in @fig-patch-examples, but this time, across multiple distinct chemical exposures:

In [ ]:
#| echo: false
comparison_movie_id = 'cb2ee929-6bf5-4742-b9a5-21675c183cf9'
comparison_movie_frames = _get_movie( comparison_movie_id, verbose = False )

In [ ]:
#| echo: false
COMPOUND_COLORS = {
    'tacpd': 'C0',
    'baclofen': 'C3',
}
COMPOUND_CMAPS = {
    'tacpd': 'managua',
    'baclofen': 'managua_r',
}

In [ ]:
#| code-fold: true
#| label: fig-patch-examples-compare
#| fig-column: page-inset-right
#| fig-cap: Example frames of patch 1 from @fig-patch-examples above, now showing evolution of calcium activity over time within the *same* patch relative to bath application of either baclofen (top row) or *t*-ACPD (bottom row). Note that, consistent with the findings from @cahill2024network, *t*-ACPD evokes a much larger response in this astrocyte patch. (A small Gaussian smoothing kernel in space and time has been applied here, to aid visualization.)

test_patch_compare = test_patches[0]

movies_compare = [
    first_movie_id,
    comparison_movie_id,
]
frame_blocks_compare = [
    movie_frames,
    comparison_movie_frames,
]

test_patch_frames_compare = []
cur_i, cur_j = test_patch_compare

for cur_frames in frame_blocks_compare:

    patch_idx_y = ( int( np.floor( patch_size_y * cur_i ) ),
                    int( np.ceil( patch_size_y * (cur_i + 1) ) ) )
    patch_idx_x = ( int( np.floor( patch_size_y * cur_j ) ),
                    int( np.ceil( patch_size_y * (cur_j + 1) ) ) )

    test_patch_frames_compare.append(
        np.array( [ 
            frame.image[0][patch_idx_y[0]:patch_idx_y[1], :][:, patch_idx_x[0]:patch_idx_x[1]]
            for frame in cur_frames
        ] )
    )

test_patch_frames_compare_filtered = [
    gaussian_filter( fs, sigma = (3., 0.6, 0.6) )
    for fs in test_patch_frames_compare
]

#

dt = movie_frames[3].t
t_intervene = 300

cmax = 70
i_frame_compare = (250, 350, 450, 550, 650, 750)

n_series = len( test_patch_frames_compare )
n_frame_compare = len( i_frame_compare )

#

with HouseStyle() as s:

    fig, axs = plt.subplots( n_patches, n_frame_compare,
        figsize = (10, 4),
        sharey = True,
    )

    for i_series in range( n_series ):

        cur_patch_frames = test_patch_frames_compare_filtered[i_series]
        cur_compound = frame_blocks_compare[i_series][0].applied_compound

        for i_t in range( n_frame_compare ):
            ax = axs[i_series, i_t]

            ax.imshow(
                cur_patch_frames[i_frame_compare[i_t], :, :],
                #
                clim = (-cmax, cmax),
                cmap = COMPOUND_CMAPS[cur_compound],
            )

            if i_series == 0:
                t_val = i_frame_compare[i_t] * dt - t_intervene
                ax.set_title( f'{t_val:+0.0f}s', fontsize = 12 )

            if i_series == (n_patches - 1) and i_t == 0:
                ax.tick_params( axis = 'x', length = 3 )
                ax.set_xticks( [-0.5, patch_size_x+0.5], ['0', f'{movie_frames[0].scale_x * patch_size_x:0.0f} µm'], ha = 'left' )
            else:
                ax.tick_params( axis = 'x', length = 0 )
                ax.set_xticks( [-0.5, patch_size_x+0.5], ['', ''] )
            
            ax.set_yticks( [] )

            if i_t == 0:
                ax.set_ylabel( f'{cur_compound}', color = COMPOUND_COLORS[cur_compound] )


    fig.patches.extend( [
        plt.Rectangle( (0.382, 0.0895), 0.53, 0.82,
            fill = True,
            color = 'k',
            alpha = 0.05,
            zorder = -1_000,
            transform = fig.transFigure,
            figure = fig,
        )
    ] )

    plt.subplots_adjust( wspace = 0.04, hspace = 0. )

We see by eye that *t*-ACPD evokes a considerably larger response in this patch, and this is reflected in the patch embeddings: while in @fig-patch-example-traces above we were able to clearly distinguish the subtle modulations induced by the introduction of baclofen, when placed on the same scale (top row) as the changes induced in this same patch by *t*-ACPD (bottom row), in @fig-patch-example-traces-compare below, the difference is stark:

In [ ]:
#| code-fold: true
#| label: fig-patch-example-traces-compare
#| column: body-outset-right
#| fig-cap: "For the astrocyte network patch depicted in @fig-patch-examples-compare, the time-evolution of three representative embedding PCs relative to the onset of bath application of baclofen (top row) or *t*-ACPD (bottom row) at *t*\_=\_0. Time shown as seconds relative to compound entering the bath (shaded box); changes shown as multiples of the baseline distribution standard deviation. (*n.b.*: a 60s boxcar filter in time has been applied to each PC, leading to blurring of activation edges.)"

from tqdm import tqdm

import atdata

import astrocytes
from astrocytes._datasets._embeddings import (
    PatchEmbeddingTrace,
)

#

## Plot params
t_intervene = 300
baseline_window = (-100, -15)
plot_window = (-100, 240)

panel_yl = (-6, 85)

#

wds_url = (
    'https://data.forecastbio.cloud/testing/patch-pc-traces/bath-application/'
    + 'bath_app-dinov3_vit7b16-pca64-smooth84.tar'
)
ds = atdata.Dataset[PatchEmbeddingTrace]( wds_url )

test_traces_compare = [ None for _ in movies_compare ]
# for trace in tqdm( ds.ordered( batch_size = None ) ):
for trace in ds.ordered( batch_size = None ):
    try:
        assert trace.metadata is not None and 'uuid' in trace.metadata
        assert trace.metadata['uuid'] in movies_compare
        
        i_movie = movies_compare.index( trace.metadata['uuid'] )

        if trace.i_patch == test_patch_compare[0] and trace.j_patch == test_patch_compare[1]:
            test_traces_compare[i_movie] = trace
            # print( 'Caught', trace.metadata['uuid'] )
        
        should_finish = True
        for x in test_traces_compare:
            if x is None:
                should_finish = False
        if should_finish:
            break

    except:
        # print( trace.metadata['uuid'], trace.i_patch, trace.j_patch )
        continue

#

causal_offset = (84 * dt) / 4.
t_rel = test_traces[0].ts - t_intervene + (causal_offset)

with HouseStyle( grids = True ) as s:

    fig, axs = plt.subplots( n_patches, 1,
        figsize = (7, 12),
        sharex = True,
        # sharey = True,
    )

    for i_series in range( n_patches ):
        ax = axs[i_series]
        cur_trace = test_traces_compare[i_series]
        cur_compound = frame_blocks_compare[i_series][0].applied_compound

        values_0_norm, mean_0, std_0 = \
            _baseline_normalize( t_rel, cur_trace.values[0, :], baseline_window )    
        values_1_norm, mean_1, std_1 = \
            _baseline_normalize( t_rel, cur_trace.values[1, :], baseline_window )
        values_2_norm, mean_2, std_2 = \
            _baseline_normalize( t_rel, cur_trace.values[4, :], baseline_window )

        filter_plot = (
            (t_rel >= plot_window[0])
            & (t_rel < plot_window[1])
        )

        ax.plot( t_rel[filter_plot], values_0_norm[filter_plot], 'C0-', label = 'PC 1' )
        ax.plot( t_rel[filter_plot], values_1_norm[filter_plot], 'C2-', label = 'PC 2' )
        ax.plot( t_rel[filter_plot], values_2_norm[filter_plot], 'C3-', label = 'PC 5' )
        
        yl = panel_yl

        ax.fill_between( [0, plot_window[1]], yl[0], yl[1], color = 'k', alpha = 0.05, linewidth = 0 )

        ax.plot( plot_window, [0, 0], 'k-', linewidth = 1 )
        ax.plot( [plot_window[0], plot_window[0]], yl, 'k-', linewidth = 1 )

        ax.set_xticks( [-90, 0, 60, 120, 180, 240] )
        # ax.set_yticks( [0, 5, 10] )
        ax.set_xlim( plot_window )
        ax.set_ylim( yl )

        ax.set_ylabel( f'{cur_compound}', color = COMPOUND_COLORS[cur_compound] )

        if i_series == (n_patches - 1):
            ax.set_xlabel( 'Time (s)' )
            # ax.set_ylabel( '∆ (baseline SD)' )

            plt.legend( fontsize = 12, loc = 'upper left' )

            ax.text( 5, 72.5, '+ Drug', alpha = 0.5 )

    fig.subplots_adjust( hspace = 0.05 )

#### Consistent pharmacology-specific profiles
The fact that these feature axes within patch embedding--space separate responses to the two compounds is not unique to this patch or this cortical slice: in fact, we can average across the responses in all patches in all recordings throughout the entire OpenAstrocytes bath application dataset, in order to view the unique profiles of dynamic responses to baclofen (red) and *t*-ACPD (blue), and quantify those responses' heterogeneity:

In [ ]:
#| echo: false

#

from typing import (
    Callable,
)

#

def boot_sample_traces( Xs: NDArray, axis = 0 ) -> NDArray:

    sample_idx = np.random.randint( Xs.shape[axis], size = (Xs.shape[axis]) )
    return Xs[sample_idx, :]

def boot_traces( Xs: NDArray,
            n: int = 200,
            axis: int = 0,
            verbose: bool = False
        ) -> NDArray:
    
    ret = np.zeros( (n,) + Xs.shape )
    it = tqdm( range( n ) ) if verbose else range( n )

    for i in it:
        ret[i] = boot_sample_traces( Xs, axis = axis )
    return ret

def boot_stat( Xs: NDArray, f: Callable[[NDArray], NDArray],
            n: int = 200,
            axis: int = 0,
            verbose: bool = False,
        ) -> NDArray:

    it = tqdm( range( n ) ) if verbose else range( n )
    
    boot_stats = np.array( [ f( boot_sample_traces( Xs, axis = axis ) )
                             for i in it ] )
    return boot_stats

In [ ]:
#| echo: false

#

wds_url = (
    'https://data.forecastbio.cloud/testing/patch-pc-traces/bath-application/'
    + 'bath_app-dinov3_vit7b16-pca64-smooth84.tar'
)
ds = atdata.Dataset[PatchEmbeddingTrace]( wds_url )

#

compound_aliases = {
    'baclofen': [
        'baclofen',
        'bacloffen',
    ],
    'tacpd': [
        'tacpd',
    ],
}

compound_traces: dict[str, list[PatchEmbeddingTrace]] = {
    c: []
    for c in compound_aliases.keys()
}

MIN_CONCENTRATION = 50. #uM

# for trace in tqdm( ds.ordered( batch_size = None ) ):
for trace in ds.ordered( batch_size = None ):
    try:
        assert trace.metadata is not None, \
            'No metadata for trace'
        assert 'compound' in trace.metadata and 'concentration' in trace.metadata, \
            'Missing metadata keys for trace'
        
        if trace.metadata['concentration'] >= MIN_CONCENTRATION:
            for k, vs in compound_aliases.items():
                if trace.metadata['compound'].lower() in vs:
                    compound_traces[k].append( trace )

    except Exception as e:
        # print( 'bad sample trace' )
        continue

In [ ]:
#| code-fold: true
#| label: fig-patch-pcs-population
#| column: body-outset
#| fig-cap: Average changes in the first eight principal components of patch embeddings relative to application of baclofen (red) or *t*-ACPD (blue). \\newline Time shown as seconds relative to compound entering the bath (shaded box); changes shown as multiples of the baseline distribution standard deviation. Error corridors show population mean ± trace-wise bootstrapped 95% confidence interval of the mean.

pcs_show_testing = (0, 1, 2, 3, 4, 5, 6, 7)

n_pcs_show = len( pcs_show_testing )

with HouseStyle( grids = True ) as s:

    fig, axs = plt.subplots( n_pcs_show // 2, 2,
        figsize = (11, 12),
        sharex = True,
    )

    for i_seq, i_pc in enumerate( pcs_show_testing ):
        i_ax = i_seq // 2
        j_ax = i_seq % 2
        ax = axs[i_ax, j_ax]

        ts_rel = trace.ts - t_intervene + causal_offset
        filter_baseline = (
            (ts_rel >= baseline_window[0])
            & (ts_rel < baseline_window[1])
        )

        raw_values = {
            c: np.array( [
                _baseline_normalize(
                    ts_rel,
                    trace.values[i_pc, :],
                    baseline_window,
                )[0]
                for trace in traces
            ] )
            for c, traces in compound_traces.items()
        }

        middle_trace = {
            c: np.mean( vs, axis = 0 )
            for c, vs in raw_values.items()
        }

        # print( { c: vs.shape for c, vs in raw_values.items() } )

        middle_boots = {
            c: boot_stat( vs, lambda x: np.mean( x, axis = 0 ), axis = 0,
                n = 200,
                verbose = False,
            )
            for c, vs in raw_values.items()
        }
        # print( { c: vs.shape for c, vs in middle_boots.items() } )
        middle_low = {
            c: np.quantile( vs, 0.025, axis = 0 )
            for c, vs in middle_boots.items()
        }
        middle_high = {
            c: np.quantile( vs, 0.975, axis = 0 )
            for c, vs in middle_boots.items()
        }

        filter_plot = (
            (ts_rel >= plot_window[0])
            & (ts_rel < plot_window[1])
        )

        #

        for c in middle_trace.keys():
            ax.plot( ts_rel[filter_plot], middle_trace[c][filter_plot], '-',
                color = COMPOUND_COLORS[c]
            )
            ax.fill_between(
                ts_rel[filter_plot],
                middle_low[c][filter_plot],
                middle_high[c][filter_plot],
                color = COMPOUND_COLORS[c],
                alpha = 0.2,
                linewidth = 0.5,
            )

        xl = plot_window
        yl = ax.get_ylim()
        # yl = (-6, 6)

        ax.plot( xl, [0, 0], 'k-', linewidth = 1 )

        ax.fill_between( [0, xl[1]], yl[0], yl[1],
            color = 'k',
            alpha = 0.05,
            linewidth = 0,
        )

        ax.set_xlim( xl )
        ax.set_ylim( yl )

        ax.set_ylabel( f'PC {i_pc + 1}' )
        ax.set_xticks( [-90, 0, 60, 120, 180, 240] )

        if j_ax == 0 and i_ax == len( pcs_show_testing ) // 2:
            ax.set_xlabel( 'Time (s)' )
        
        if j_ax == 0 and i_ax == 0:
            ax.text( 5, -1.5, '+ Drug',
                alpha = 0.5,
                va = 'center',
            )
    
    fig.subplots_adjust( wspace = 0.23, hspace = 0.09 )

Amazingly, we see that these feature axes capture *consistent* differences in the evoked imaging features between the two compounds, across a large array of different individual mice, slices, and experiments. This suggests that the image feature axes modulated by applied pharmacology are not arbitrary divisions in image-space, but are instead capturing regularities in the ways that these compounds' impacts on astrocytes consistently change intracellular calcium computations in response.[This concords with our previous work, in which we found one specific image feature of astrocyte calcium dynamics---the extent to which individual calcium events *propagate* in space---defined a subset of calcium excitations carrying its own separable information content about a distinct chemical exposure, in that case to extracellular glutamate [@cahill2024network].]{.aside}

#### Decoding chemical identity
The question then arises: *Are these consistent differences robust enough to **decode** the compound applied to a particular patch of astrocyte network?* We can test this by using a simple (generalized) linear readout^[Here, *via* [support vector classification](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).] of our PC-space to classify the applied compound. To understand in greater detail the way this information content about the applied compound dynamically unfolds, we'll restrict ourselves to looking at decoding from the activity at a single time-point after compound application, and then vary across all time-points within the experiment. We'll also restrict ourselves, for simplicity, to decoding the compound identity from *one single* ~60µm patch; this makes the decoding problem its most difficult flavor (rather than giving ourselves additional information by, for example, aggregating across patches within the same slice), in order to avoid ceiling effects.

Evaluating this single-patch compound decoding performance across time reveals some interesting patterns:

In [ ]:
#| echo: false

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

#

compound_labels = {
    'baclofen': 0,
    'tacpd': 1,
}

window_decode = (-60, 240)

#

ts_rel = compound_traces['baclofen'][0].ts - t_intervene + causal_offset
filter_decode = (
    (ts_rel >= window_decode[0])
    & (ts_rel < window_decode[1])
)

idx_iter = np.where( filter_decode )[0]
idx_iter = idx_iter[::20]

n_pc_try = [4, 8, 12, 16]
n_cv = 20

acc_cv = [ np.zeros( (idx_iter.shape[0], n_cv) )
           for _ in n_pc_try ]
acc_cv_shuffle = np.zeros( (idx_iter.shape[0], n_cv) )

for i_pcs_seq, n_pc_use in enumerate( n_pc_try ):

    # for i_t_seq, i_t in tqdm( enumerate( idx_iter ), total = idx_iter.shape[0] ):
    for i_t_seq, i_t in enumerate( idx_iter ):

        features_cur = []
        labels_cur = []
        for c, traces in compound_traces.items():
            for trace in traces:
                features_cur.append( trace.values[:n_pc_use, i_t] )
                labels_cur.append( compound_labels[c] )
        features_cur = np.array( features_cur )
        labels_cur = np.array( labels_cur )

        n_train = int( 0.8 * labels_cur.shape[0] )
        n_test = labels_cur.shape[0] - n_train
        filter_train = np.r_[np.ones( (n_train,) ), np.zeros( (n_test,) )]

        for i_cv in range( n_cv ):
            filter_train = np.random.permutation( filter_train ).astype( bool )
            filter_test = ~filter_train

            classifier = SVC( gamma = 'scale' )
            # classifier = LogisticRegression()

            classifier.fit( features_cur[filter_train, :], labels_cur[filter_train] )
            labels_hat = classifier.predict( features_cur[filter_test, :] )
            acc_cur = np.sum( labels_hat == labels_cur[filter_test] ) / np.sum( filter_test )
            acc_cv[i_pcs_seq][i_t_seq, i_cv] = acc_cur
        
        #

        # Most favorable possible comparison for perm
        if n_pc_use == n_pc_try[-1]:

            labels_cur_shuffled = np.random.permutation( labels_cur )

            n_train = int( 0.8 * labels_cur_shuffled.shape[0] )
            n_test = labels_cur_shuffled.shape[0] - n_train
            filter_train = np.r_[np.ones( (n_train,) ), np.zeros( (n_test,) )]

            for i_cv in range( n_cv ):
                filter_train = np.random.permutation( filter_train ).astype( bool )
                filter_test = ~filter_train

                svc = SVC( gamma = 'auto' )
                svc.fit( features_cur[filter_train, :], labels_cur_shuffled[filter_train] )
                labels_hat = svc.predict( features_cur[filter_test, :] )
                acc_cur = np.sum( labels_hat == labels_cur_shuffled[filter_test] ) / np.sum( filter_test )
                acc_cv_shuffle[i_t_seq, i_cv] = acc_cur

In [ ]:
#| code-fold: true
#| label: fig-decoding
#| column: body-outset-right
#| fig-cap: Accuracy of classifying applied pharmacology (baclofen *vs.* *t*-ACPD) from embeddings of a single astrocyte network patch, as a function of time after application of the compound. Colored bands indicate bootstrapped confidence interval of accuracy across cross-validation replicates. Each trace color corresponds to a distinct number of PCs included in the decoder. The black trace indicates a null condition in which the compound labels were permuted at each time point, but decoding was performed otherwise in the same manner---*i.e.*, the random chance level for decoding performance.
with HouseStyle( grids = True ) as s:
    fig, ax = plt.subplots( figsize = (8, 5) )

    #

    for i_seq, (n_pc_cur, acc_cv_cur) in enumerate( zip( n_pc_try, acc_cv ) ):
        middle_trace = np.mean( acc_cv_cur.T, axis = 0 )
        middle_boots = boot_stat( acc_cv_cur.T, lambda x: np.mean( x, axis = 0 ), axis = 0,
            n = 1_000,
            verbose = False
        )
        middle_low = np.quantile( middle_boots, 0.005, axis = 0 )
        middle_high = np.quantile( middle_boots, 0.995, axis = 0 )

        ax.plot( ts_rel[idx_iter], middle_trace, label = f'{n_pc_cur} PCs', zorder = 200 - i_seq * 10 )
        ax.fill_between( ts_rel[idx_iter], middle_low, middle_high,
            alpha = 0.2,
            zorder = 199 - i_seq * 10,
        )

    #

    middle_trace = np.mean( acc_cv_shuffle.T, axis = 0 )

    middle_boots = boot_stat( acc_cv_shuffle.T, lambda x: np.mean( x, axis = 0 ), axis = 0,
        n = 1_000,
        verbose = False
    )
    middle_low = np.quantile( middle_boots, 0.005, axis = 0 )
    middle_high = np.quantile( middle_boots, 0.995, axis = 0 )

    ax.plot( ts_rel[idx_iter], middle_trace, color = 'k', label = 'Permuted', zorder = -100 )
    ax.fill_between( ts_rel[idx_iter], middle_low, middle_high,
        color = 'k',
        alpha = 0.2,
        zorder = 20
    )

    xl = window_decode
    # yl = ax.get_ylim()
    yl = (0.36, 1.04)

    ax.plot( xl, [0, 0], 'k-', linewidth = 1 )

    ax.fill_between( [0, xl[1]], yl[0], yl[1],
        color = 'k',
        alpha = 0.05,
        linewidth = 0,
        zorder = 10,
    )


    ax.text( 5, 0.75, '+ Drug', alpha = 0.5, va = 'center' )

    ax.set_xlabel( 'Time (s)' )
    ax.set_ylabel( f'Decoding accuracy' )

    plt.legend( fontsize = 12 )

    # s.label(
    #     xaxis_y = 0.36,
    #     data_xlim = (-60, 240),
    # )
    ax.set_xticks( [-60, 0, 60, 120, 180, 240] )
    ax.set_xlim( xl )
    ax.set_ylim( yl )

Not only are we able to decode quite well the identity of the applied compound from a *single patch* of the astrocyte network, but it is clear that the particular richness of these imaging features is necessary in order to tease the compound differences apart: while decoding from only 4 PCs gives performance above chance, increasing to 12 or 16 included PCs dramatically increases peak decoding accuracy. This indicates that what is at play here in distinguishing systematically between the two compounds is not a simple matter of intensity, or some other single aspect of the imaging: rather, a sparse but rich *subspace* of the overall DINOv3 patch embedding--space contains the information content needed to determine applied chemical identity from the induced changes in the astrocyte network. This may indicate that these image features contain a nontrivial reflection, *via* our astrocyte calcium activity observable, of underlying differences in physiological processes recruited by the two distinct pathways engaged by baclofen and *t*-ACPD (*i.e.*, GABA~B~ vs mGluR2/3 receptors).


### Discussion

This brief foray into analyzing the OpenAstrocytes data demonstrates the remarkable richness present in astrocyte calcium dynamics, as well as the amazing potential that lies in making this data readily available in a format that slots directly into today's AI-driven biological data science workflows.

We're particularly interested in highlighting three key patterns we've seen here:

#### Generalist AI features contain specialized information
While DINOv3 [@siméoni2025dinov3] was trained on a very broad image corpus, we were able to see in the structure of embeddings inferred from the OpenAstrocyte bath application data in particular two major, distinct contributors: (1) anatomical features of local astrocyte networks, as well as (2) physiological features of the dynamics of intracellular astrocyte calcium. These two contributions were reflected in small subspaces of the full 4096-dimensional patch embedding space, indicating, as expected, that while there is a rich spectrum of image structure present in astrocyte network calcium, this activity is constrained to a smaller, more regular subspace.

The fact that this large model, trained in a largely-unbiased way on principally non-biological images, has subspaces that are so strongly reflective of detailed image features for this sub-field, is at this point unsurprising but still remarkable. This finding (and many others like it applying large AI models across other specialist domains of science that have emerged in recent years) has massive implications for the potential utility of these generalist large models to biological data science.

#### Heterogeneity of responses across individual fields
It is amazing to see *consistency* in the responses of particular subspaces to particular drugs, as demonstrated in @fig-patch-pcs-population and @fig-decoding; however, as we move forward in understanding the potential lying in these models to find new biology, we're very excited to dig into what underlies the *differences* between responses in specific small pieces of the *same* slice of brain tissue. That neighboring ~60µm patches can exhibit drastically diverging calcium-feature responses to the same applied stimulus suggests a heterogeneity in the processing apparatus of constituent parts of the astrocyte network that is hinted at in the field but as-yet not fully elucidated. We think there's a lot more to know about astrocytes---and brains---hidden in the characterization of these response differences.

#### Decoding performance from rich features
We saw great chemical decoding performance from a *single frame* of *one ~60µm patch* of a cortical slice---and this is a *pessimistic* estimate, given how many patches were unresponsive due to differences in expression of the calcium indicator! Similar to the above, there's a lot to tease apart in the richness of the heterogeneity we saw, particularly in the structure time-evolution. Our previous work in [@cahill2024network] indicated that astrocyte networks integrate physiologic information across minutes, and it's fantastic to see that similarly reflected in the time-course of our chemical identity decoding here.

While we didn't need to include *all* of the DINOv3 embedding features to get good decoding, we needed a few. We find this pattern of results the most exciting possibility, as it suggests that the decoding performance isn't arising from some trivial feature (like increased intensity), but instead from a richer combination of features that could generalize to more interesting predictive tasks. We are interested to see how this rich structure reflects the intricate machinery astrocyte networks have for integrating diverse chemical signals when probed with a larger swath of pharmacological space.

 

------------------------------------------------------------------------

## Looking ahead

The OpenAstrocytes corpus is one step in our larger thrust into providing open tools to understand the dynamics of astrocyte physiology, as well as our approach within Forecast to use this incredible science to understand how our chemical tools change brain networks.

#### Building on `atdata` {#sec-future-atdata}
`atdata`'s addition of sample-typing functionality to WebDatasets already helps for keeping streamed AI datasets sane; but, where keeping track of sample types really shines is when we register *mappings* between types. As an example, in the the `astrocytes` library accompanying this pub, we have snippets like:

``` python
@atdata.lens
def from_generic( s: ts.Frame ) -> 'BathApplicationFrame':
    # 1. Performs extraction of features and
    # validation  from unstructured metadata ...
    return BathApplicationFrame(
        # 2. Wraps into a structured sample class
    )
```

Under the hood, the `atdata.lens` decorator keeps track of the full registry of lenses between sample schema types, and the way they compose. Because lenses can have a very nice compositional behavior [@nlab:lens], this design enables us, as we're building AI models from a bunch of different sources, to easily and automatically aggregate across our diverse hive of WebDatasets, built across various modalities and experiments---and, to take advantage of the natural ways these data can be compatibly merged.

The full `atdata` ecosystem is architected to go one step further, placing metadata for type schemas of scientific datasets---and the code for these lenses between them!---in a public, decentralized repository on [ATProto](https://atproto.com), the protocol layer that lies at the heart of the [Bluesky](https://bsky.social/about) social network. This allows AI coding agents on data science tasks to automatically leverage domain knowledge of structured semantic relationships to decide what data to pull from for a given task.[More on `atdata` soon, including our ATProto appview for the full distributed dataset network with MCP support---[reach out to sign up for updates](mailto:hello@forecast.bio) in order to hear first as we push more features to the public release!]{.aside}

#### Loving neurons, too
Of course, the brain *also* is one-third neurons---a fact we certainly wouldn't want to forget. This is why we're building out the OpenAstrocytes repository with newly-collated data from the Allen Brain Observatory [@de2023sharing], also formatted with `atdata` for easy use with existing AI workflows.

We're continuing to collate this fantastic dataset's movies of raw two-photon calcium imaging from neurons in visual cortex, in order to make this phenomenal source of neurobiological imaging data more accessible for augmenting workflows like AI fine-tuning; since neuronal and astrocytic imaging share tremendous overlap in latent image statistics, we anticipate that combining large datasets of similar dynamic imaging from the two cell types will provides immense benefits to the study of each.

#### A new kind of data
Here at Forecast, we're building out a new experimental dataset taken not from , but instead from 3D *in vitro* systems built from human induced pluripotent stem cells. Human astrocyte dynamics are a near-completely unexplored frontier within neuroscience; we are extremely excited about the potential lying within this system, as we build out OpenAstrocytes in the next few months with more and more data taken from this completely blank canvas. Stay tuned ;).

 

------------------------------------------------------------------------

 

#### Acknowledgements
We'd like to thank **Dr. Michelle Cahill** and **Dr. Michael Reitman** for driving the original experimental work and data collection for the images that went into the initial release of OpenAstrocytes presented here.

Support for the production of OpenAstrocytes at Forecast was generously provided by the Special Initiatives division of the [**Astera Institute**](https://astera.org/).

#### Citation
Please reference this work in BibTeX as:

``` bibtex
@article{levesque2025openastrocytes,
  author = {Maxine Levesque and Kira Poskanzer},
  title = {OpenAstrocytes},
  journal = {Forecast Research},
  year = {2025},
  note = {https://forecast.bio/research/open-astrocytes/},
}
```

#### AI use
No language model tools were used to produce any of the code for or writing in this pub.

#### Copyright
Copyright © 2025 Forecast Bio, Inc. The contents of this article are licensed under the Creative Commons [CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/) license.

 

------------------------------------------------------------------------

 

#### References

::: {#refs}
:::